<a href="https://colab.research.google.com/github/AnnaZhuravleva/avtobreja/blob/master/Sentiment_for_project_refactor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install innvestigate

In [0]:
!pip install tensorflow==1.14.0

In [24]:
!pip install tqdm

In [0]:
!wget http://vectors.nlpl.eu/repository/11/180.zip
!unzip 180.zip

--2019-12-21 13:16:12--  http://vectors.nlpl.eu/repository/11/180.zip
Resolving vectors.nlpl.eu (vectors.nlpl.eu)... 129.240.189.225
Connecting to vectors.nlpl.eu (vectors.nlpl.eu)|129.240.189.225|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 484452285 (462M) [application/zip]
Saving to: ‘180.zip.1’

180.zip.1             3%[                    ]  14.97M   272KB/s    eta 40m 36s^C
Archive:  180.zip
replace README? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace meta.json? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model.bin? [y]es, [n]o, [A]ll, [N]one, [r]ename: n
replace model.txt? [y]es, [n]o, [A]ll, [N]one, [r]ename: n


In [1]:
import innvestigate
import keras.backend
import keras.models
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import pickle
import sys
import time

from collections import Counter
from gensim.models import KeyedVectors
from itertools import chain
from innvestigate.utils.tests.networks import base as network_base
from matplotlib import cm, transforms
from sklearn.model_selection import train_test_split
from tqdm import tqdm

Using TensorFlow backend.
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/usr/local/lib/python3.6/dist-packages/tensorflow/python/framework/dtypes.py:519: FutureWarn

## Оценка тональности с помощью CNN

В качестве входных представлений будем использовать word2vec для лемм с POS-тегами UD.
Архитектура классификатора примерно воспроизводит описанную в [статье Arras et al. 2017](http://www.aclweb.org/anthology/W16-1601); а для визуализации воспользуемся библиотекой [iNNvestigate](https://github.com/albermax/innvestigate).

Картинка про классификатор и оценку значимости входных слов ([источник](https://doi.org/10.1371/journal.pone.0181142.g001)):
<img src="https://camo.githubusercontent.com/ba37f37fdbb90ccd76f1c4bf399e0cb8ddbc66f0/68747470733a2f2f692e696d6775722e636f6d2f595144665335502e706e67"/>

### CNN functions

In [0]:
def max_length(texts):
    return max(len(t) for t in texts)
    
def to_one_hot(y):
    return keras.utils.to_categorical(y, 2)

In [0]:
def load_dataset(w2v_model, lines, MAX_LEN, num_examples=None):
    """
    embedds a list of texts with a w2v model

    :param w2v_model: gensim KeyedVectors model
    :param lines: list of lists of str, texts of words (pre-processed to be model-compatible)
    :param MAX_LEN: int, maximal text length for padding
    :param num_examples: int, number of texts to add to dataset, optional, default None 

    :return x_tensor: np.ndarray, embedded texts
    :return vocab: counter, how many times words occured in texts
    """
    embedding_dim = w2v_model.vector_size
    prep = lines[:num_examples]
    vocab = Counter()
    x_tensor = np.zeros((len(prep), MAX_LEN, embedding_dim))
    for i, text in enumerate(prep):
        for j, w in enumerate(text):
            try:
                x_tensor[i, j, :] = w2v_model[w]
            except KeyError:
                pass
        vocab[w] += 1
    return x_tensor, vocab

In [0]:
def make_dataset_for_training_cnn(w2v_model, texts, scores):
  """
  vectorizes dataset from a list of texts and scores (do not give more than 2000, as more eats all RAM)

  :param w2v_model: gensim KeyedVectors model
  :param texts: list of lists of str, texts of words
  :param scores: list of int, target text scores

  :return MAX_LEN: int, maximal text length for padding
  :return scores_train: list of int, traing scores
  :return scores_val: list of int, validation scores
  :return texts_train: list of lists of str, traing texts
  :return texts_val: list of lists of str, validation texts
  :return input_tensor_train: np.ndarray, embedded traing texts
  :return inp_vocab_train: counter, traing texts vocabulary
  :return input_tensor_val: np.ndarray, embedded validation texts
  :return inp_vocab_val: counter, validation texts vocabulary
  """
  if len(texts) != len(scores):
    raise ValueError(f"expected texts and scores of the same length but got lengths {len(texts)} and {len(scores)}")
  scores = np.array(scores)
  binary_scores = scores > 5.
  binary_scores = binary_scores.astype(int)
  scores_train, scores_val, texts_train, texts_val = train_test_split(binary_scores, texts, test_size=0.3, shuffle=False)
  MAX_LEN = max(max_length(texts_train), max_length(texts_val))
  input_tensor_train, inp_vocab_train = load_dataset(w2v_model, texts_train, MAX_LEN)
  input_tensor_val, inp_vocab_val = load_dataset(w2v_model, texts_val, MAX_LEN)
  return MAX_LEN, scores_train, scores_val, texts_train, texts_val, input_tensor_train, inp_vocab_train, input_tensor_val, inp_vocab_val

In [0]:
def build_network(max_len, voc_size, embedding_dim, output_n, activation=None, dense_unit=256, dropout_rate=0.25):
    """
    builds a keras cnn structure

    :param max_len: int, sequence length
    :param voc_size: int, number of words in vocabulary
    :param embedding_dim: int, dimensionality of w2v model
    :param output_n: int, number of classes
    :param activation: pre-trained weights, optional, default None
    :param dense_unit: int, optional, dimensionality of the dense unit 
    :param dropout_rate: float, optional, default=0.25

    :return: keras nn
    """
    if activation:
        activation = "relu"

    net = {}
    net["in"] = keras.Input(shape=[1, max_len, embedding_dim])
    net["conv"] = keras.layers.Conv2D(filters=100, kernel_size=(1,2), strides=(1, 1), padding='valid')(net["in"])
    net["pool"] = keras.layers.MaxPooling2D(pool_size=(1, max_len - 1), strides=(1,1))(net["conv"])
    net["out"] = network_base.dense_layer(keras.layers.Flatten()(net["pool"]), units=output_n, activation=activation)
    net["sm_out"] = network_base.softmax(net["out"])


    net.update({
        "input_shape": [1, max_len, embedding_dim],
        "output_n": output_n,
    })
    return net

In [0]:
def train_model(model, input_tensor_train, input_tensor_val, scores_train, scores_val, epochs=20):
    """
    training keras nn

    :param model: keras nn
    :param input_tensor_train: np.ndarray, training set
    :param input_tensor_val: np.ndarray, validation set
    :param scores_train: list of int, traing scores
    :param scores_val: list of int, validation scores
    :param epochs: int, optional, default 20
    """
    x_train = np.expand_dims(input_tensor_train, axis=1)
    y_train = to_one_hot(scores_train)
    
    x_val = np.expand_dims(input_tensor_val, axis=1)
    y_val = to_one_hot(scores_val)

    model.compile(loss='categorical_crossentropy',
                  optimizer=keras.optimizers.Adam(),
                  metrics=['accuracy'])

    history = model.fit(x_train, y_train,
                        batch_size=256,
                        epochs=epochs,
                        verbose=1,
                        validation_data=(x_val, y_val),
                        shuffle=True)

In [0]:
def train_network(w2v_model, texts_train, texts_val, scores_train, scores_val, inp_vocab_train, inp_vocab_val, input_tensor_train, input_tensor_val, MAX_LEN, epochs=20):
  """
  trains two cnn models form split dataset

  :param w2v_model: gensim KeyedVectors model
  :param texts_train: list of lists of str, traing texts
  :param texts_val: list of lists of str, validation texts
  :param scores_train: list of int, traing scores
  :param scores_val: list of int, validation scores
  :param inp_vocab_train: counter, traing texts vocabulary
  :param inp_vocab_val: counter, validation texts vocabulary
  :param input_tensor_train: np.ndarray, training set
  :param input_tensor_val: np.ndarray, validation set
  :param MAX_LEN: int, sequence length
  :param epochs: int, optional, default 20

  :return model_without_softmax: keras nn, without classifier layer
  :return model_with_softmax: keras nn, with classifier layer
  :return embedding_dim: int, dimension of w2v
  """
  embedding_dim = w2v_model.vector_size
  inp_vocab = inp_vocab_train + inp_vocab_val
  vocab_inp_size = len(inp_vocab) + 1
  net = build_network(MAX_LEN, vocab_inp_size, embedding_dim, 2)
  model_without_softmax = keras.models.Model(inputs=net['in'], outputs=net['out'])
  model_with_softmax = keras.models.Model(inputs=net['in'], outputs=net['sm_out'])
  train_model(model_with_softmax, input_tensor_train, input_tensor_val, scores_train, scores_val, epochs)
  model_without_softmax.set_weights(model_with_softmax.get_weights())
  return model_without_softmax, model_with_softmax, embedding_dim

In [0]:
def analyze_scores(X, Y, ridx, embedding_dim, model_without_softmax, model_with_softmax, analyzers, max_len):
    """
    predicts individual word scores and text score

    :param X: np.ndarray, embedded texts tensor
    :param Y: list of int, target classes
    :param ridx: int, text id 
    :param model_without_softmax: keras nn, without classifier layer
    :param model_with_softmax: keras nn, with classifier layer
    :param embedding_dim: int, dimension of w2v
    :param analyzers: list of innvestigate analyzers
    :param max_len: int, sequence length

    :return analysis: list of list of float, word weights according to each analyzer
    :return y_hat: int, calss prediction
    """
    analysis = np.zeros([len(analyzers), 1, max_len])
    x, y = X[ridx], Y[ridx]
    t_start = time.time()
    x = x.reshape((1, 1, max_len, embedding_dim))
    presm = model_without_softmax.predict_on_batch(x)[0] #forward pass without softmax
    prob = model_with_softmax.predict_on_batch(x)[0] #forward pass with softmax
    y_hat = prob.argmax()
  
    for aidx, analyzer in enumerate(analyzers):
        a = np.squeeze(analyzer.analyze(x))
        a = np.sum(a, axis=1)
        analysis[aidx] = a
    t_elapsed = time.time() - t_start
    # print('Review %d (%.4fs)'% (ridx, t_elapsed))
    return analysis, y_hat

In [0]:
def get_sentiment_vocabulary(w2v_model, texts, scores, sentiment_scores=None, only_correct=True, epochs=10):
  """
  gets sentiment vocabulary from texts

  :param w2v_model: gensim KeyedVectors model
  :param texts: list of lists of str, texts of words
  :param scores: list of int, target text scores 
  :param sentiment_scores: counter of words to update, optional, default None
  :param only_correct: bool, if only use the weights from correct predictions, optional, default True
  :param epochs: int, cnn training param, optional, default 10

  :return: counter, words and their sentiment weights
  """
  print("__making the dataset__")
  MAX_LEN, scores_train, scores_val, texts_train, texts_val, input_tensor_train, inp_vocab_train, input_tensor_val, inp_vocab_val = make_dataset_for_training_cnn(w2v_model, texts, scores)
  print("__training the network__")
  model_without_softmax, model_with_softmax, embedding_dim = train_network(w2v_model, texts_train, texts_val, scores_train, scores_val, inp_vocab_train, inp_vocab_val, input_tensor_train, input_tensor_val, MAX_LEN, epochs=epochs)

  print("__making the analyzers__")
  methods = ['gradient', 'lrp.z', 'lrp.alpha_2_beta_1', 'pattern.attribution']
  kwargs = [{}, {}, {}, {'pattern_type': 'relu'}]
  analyzers = []
  for method, kws in zip(methods, kwargs):
      analyzer = innvestigate.create_analyzer(method, model_without_softmax, **kws)
      analyzer.fit(np.expand_dims(input_tensor_train, axis=1), batch_size=256, verbose=1)
      analyzers.append(analyzer)

  if not sentiment_scores:
    sentiment_scores = [Counter() for method in methods]
  
  for idx in tqdm(range(len(input_tensor_train)), desc="__building the sentiment vocabulary on training set__"):
    words = texts_train[idx]
    y_true = scores_train[idx]
    a, y_pred = analyze_scores(input_tensor_train, scores_train, idx, embedding_dim, model_without_softmax, model_with_softmax, analyzers, MAX_LEN)
    if only_correct and y_true == y_pred or not only_correct:
      for j, method in enumerate(methods):
        sentiment_scores[j].update(dict(zip(words, a[j].reshape(-1))))

  for idx in tqdm(range(len(input_tensor_val)), desc="__building the sentiment vocabulary on validation set__"):
    words = texts_val[idx]
    y_true = scores_val[idx]
    a, y_pred = analyze_scores(input_tensor_val, scores_val, idx, embedding_dim, model_without_softmax, model_with_softmax, analyzers, MAX_LEN)
    if only_correct and y_true == y_pred or not only_correct:
      for j, method in enumerate(methods):
        sentiment_scores[j].update(dict(zip(words, a[j].reshape(-1))))
  return sentiment_scores

## Data

### Данные
Каждый текст - строчка из токенов лемма_тег. Оценки отдельно выведены для сервиса, отдельно для еды. Мы будем строить бинарную классификацию, поэтому будем считать оценки выше 5 положительными, а 5 и ниже — отрицательными.

In [0]:
from google.colab import drive
drive.mount("/content/drive", force_remount=True)

Mounted at /content/drive


In [0]:
scored = pd.read_csv("/content/drive/My Drive/studies/HSE/prog/nlp/SentiRuEval_rest_train_lemma_POS.csv", index_col=0)
scored.tail()

,food,service,text,lemma_POS
19029,2,3,"С тех пор, как побывала последний раз, мнение ...","с_ADP тот_DET пора_NOUN ,_PUNCT как_SCONJ быва..."
19030,1,1,Обнаруженный в салате таракан очень поразил!!!...,обнаружить_VERB в_ADP салат_NOUN таракан_NOUN ...
19031,6,5,Забавное заведение. Симпатичный светлый интерь...,забавный_ADJ заведение_NOUN ._PUNCT симпатичны...
19032,8,9,Поздравляем администрацию ресторана Навруз с Н...,поздравлять_VERB администрация_NOUN ресторан_N...
19033,8,8,Очень нравиться интерьер. Отличный гриль.Карта...,очень_ADV нравиться_VERB интерьер_NOUN ._PUNCT...


In [0]:
texts = list(scored["lemma_POS"].apply(lambda s: s.split()))

In [0]:
w2v_model = KeyedVectors.load_word2vec_format('model.bin', binary=True)

/usr/local/lib/python3.6/dist-packages/smart_open/smart_open_lib.py:402: UserWarning: This function is deprecated, use smart_open.open instead. See the migration notes for details: https://github.com/RaRe-Technologies/smart_open/blob/master/README.rst#migrating-to-the-new-open-function
  'See the migration notes for details: %s' % _MIGRATION_NOTES_URL


## Food


In [0]:
scores = scored["food"].apply(float)

In [15]:
sent = get_sentiment_vocabulary(w2v_model, texts[:2000], scores[:2000], epochs=5)

__making the dataset__
__training the network__






Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 1400 samples, validate on 600 samples
Epoch 1/1
1400/1400 [==============================] - 10s 7ms/step - loss: 1.0096 - acc: 0.6329 - val_loss: 0.7459 - val_acc: 0.7550
__making the analyzers__
Instructions for updating:
ksizes is deprecated, use sizes instead


/usr/local/lib/python3.6/dist-packages/innvestigate/analyzer/base.py:130: RuntimeWarning: This analyzer does not need to be trained. Still fit() is called.
  " Still fit() is called.", RuntimeWarning)


Epoch 1/1
6/6 [==============================] - 13s 2s/step - loss: 2.0000 - broadcast_1_loss: 1.0000 - broadcast_2_loss: 1.0000


  0%|          | 0/1400 [00:00<?, ?it/s]

__building the sentiment vocabulary on training set__


  0%|          | 2/600 [00:00<00:30, 19.41it/s]

__building the sentiment vocabulary on validation set__


100%|██████████| 600/600 [00:30<00:00, 19.71it/s]


In [22]:
sent[0]['отличный_ADJ']

0.014342761598527431

In [17]:
sent[0]['ужасный_ADJ']

-0.8887263648211956

In [18]:
sent[0]['хорошо_ADV']

-0.5300868730992079

In [19]:
sent[0]['плохо_ADV']

-0.8013026644475758

In [25]:
for i in range(2000,len(texts),2000):
  print(f"____________________{i/2000}_________________________")
  if i + 2000 > len(texts):
    sent = get_sentiment_vocabulary(w2v_model, texts[i:], scores[i:], sent, epochs=5)
  else:
    sent = get_sentiment_vocabulary(w2v_model, texts[i:i+2000], scores[i:i+2000], sent, epochs=5)
  with open('/content/drive/My Drive/studies/HSE/prog/nlp/sent_food.pickle', 'wb') as f:
    pickle.dump(sent, f)

____________________2.0_________________________
__making the dataset__
__training the network__
Train on 1400 samples, validate on 600 samples
Epoch 1/5
1400/1400 [==============================] - 12s 8ms/step - loss: 1.4295 - acc: 0.6557 - val_loss: 0.9141 - val_acc: 0.5533
Epoch 2/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.6709 - acc: 0.7214 - val_loss: 0.7981 - val_acc: 0.7600
Epoch 3/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.4451 - acc: 0.8093 - val_loss: 0.6610 - val_acc: 0.6800
Epoch 4/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.3329 - acc: 0.8529 - val_loss: 0.6623 - val_acc: 0.7717
Epoch 5/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.2581 - acc: 0.8843 - val_loss: 0.5781 - val_acc: 0.7283
__making the analyzers__


/usr/local/lib/python3.6/dist-packages/innvestigate/analyzer/base.py:130: RuntimeWarning: This analyzer does not need to be trained. Still fit() is called.
  " Still fit() is called.", RuntimeWarning)


Epoch 1/1
6/6 [==============================] - 15s 3s/step - loss: 2.0000 - broadcast_3_loss: 1.0000 - broadcast_4_loss: 1.0000


  0%|          | 0/1400 [00:00<?, ?it/s]

__building the sentiment vocabulary on training set__



...

  0%|          | 0/723 [00:00<?, ?it/s]

__building the sentiment vocabulary on training set__



  0%|          | 1/311 [00:00<00:43,  7.16it/s]

__building the sentiment vocabulary on validation set__



100%|██████████| 311/311 [00:43<00:00,  7.17it/s]


In [26]:
sent[0]['отличный_ADJ']

11.635256052250043

In [27]:
sent[0]['ужасный_ADJ']

5.327932534972206

In [28]:
sent[0]['хорошо_ADV']

-10.924667179817334

In [29]:
sent[0]['плохо_ADV']

-2.546172444941476

In [0]:
with open('/content/drive/My Drive/studies/HSE/prog/nlp/sent_food.pickle', 'rb') as f:
  sent = pickle.load(f)

In [35]:
sent[0]['плохо_ADV']

-2.546172444941476

In [36]:
sent[0]['хорошо_ADV']

-10.924667179817334

## Service


In [0]:
import warnings
warnings.filterwarnings("ignore")

In [0]:
scores = scored["service"].apply(float)

In [38]:
sent = get_sentiment_vocabulary(w2v_model, texts[:2000], scores[:2000], epochs=5)

__making the dataset__
__training the network__
Train on 1400 samples, validate on 600 samples
Epoch 1/5
1400/1400 [==============================] - 13s 10ms/step - loss: 0.9575 - acc: 0.6579 - val_loss: 0.6883 - val_acc: 0.6717
Epoch 2/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.5450 - acc: 0.7521 - val_loss: 0.6064 - val_acc: 0.7783
Epoch 3/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.3737 - acc: 0.8436 - val_loss: 0.5578 - val_acc: 0.7333
Epoch 4/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.2947 - acc: 0.8807 - val_loss: 0.5454 - val_acc: 0.7883
Epoch 5/5
1400/1400 [==============================] - 11s 8ms/step - loss: 0.2155 - acc: 0.9200 - val_loss: 0.5049 - val_acc: 0.7850
__making the analyzers__
Epoch 1/1
6/6 [==============================] - 16s 3s/step - loss: 2.0000 - broadcast_21_loss: 1.0000 - broadcast_22_loss: 1.0000


__building the sentiment vocabulary on training set__: 100%|██████████| 1400/1400 [01:26<00:00, 16.19it/s]
__building the sentiment vocabulary on validation set__: 100%|██████████| 600/600 [00:33<00:00, 18.22it/s]


In [39]:
sent[0]['отличный_ADJ']

-14.341200320981443

In [40]:
sent[0]['ужасный_ADJ']

1.9018306899815798

In [41]:
sent[0]['хорошо_ADV']

1.343268582597375

In [42]:
sent[0]['плохо_ADV']

0.4736492922529578

In [0]:
for i in range(2000,len(texts),2000):
  print(f"____________________{i/2000}_________________________")
  if i + 2000 > len(texts):
    sent = get_sentiment_vocabulary(w2v_model, texts[i:], scores[i:], sent, epochs=5)
  else:
    sent = get_sentiment_vocabulary(w2v_model, texts[i:i+2000], scores[i:i+2000], sent, epochs=5)
  with open('/content/drive/My Drive/studies/HSE/prog/nlp/sent_service.pickle', 'wb') as f:
    pickle.dump(sent, f)

____________________1.0_________________________
__making the dataset__
__training the network__
Train on 1400 samples, validate on 600 samples
Epoch 1/5
1400/1400 [==============================] - 14s 10ms/step - loss: 1.2474 - acc: 0.6200 - val_loss: 0.6655 - val_acc: 0.7483
Epoch 2/5
1400/1400 [==============================] - 12s 9ms/step - loss: 0.6939 - acc: 0.6593 - val_loss: 0.5367 - val_acc: 0.7700
Epoch 3/5
1400/1400 [==============================] - 12s 9ms/step - loss: 0.4986 - acc: 0.7979 - val_loss: 0.4945 - val_acc: 0.7917
Epoch 4/5
1400/1400 [==============================] - 12s 9ms/step - loss: 0.3314 - acc: 0.8564 - val_loss: 0.5072 - val_acc: 0.7600
Epoch 5/5
1400/1400 [==============================] - 12s 9ms/step - loss: 0.2231 - acc: 0.9129 - val_loss: 0.4638 - val_acc: 0.8117
__making the analyzers__
Epoch 1/1
6/6 [==============================] - 18s 3s/step - loss: 2.0000 - broadcast_23_loss: 1.0000 - broadcast_24_loss: 1.0000


__building the sentiment vocabulary on training set__:  48%|████▊     | 675/1400 [00:51<00:44, 16.34it/s]

In [0]:
sent[0]['отличный_ADJ']

In [0]:
sent[0]['ужасный_ADJ']

In [0]:
sent[0]['хорошо_ADV']

In [0]:
sent[0]['плохо_ADV']